In [1]:
import cellpylib as cpl
import cellpylib3d as cpl3d
from collections import defaultdict
import numpy as np
from random import choice
from cellular.cellular import update_cellular
import torch
import argparse
import SimpleITK as sitk
from skimage.measure import label, regionprops
import cv2
import math
Organ_List = {'liver': [6], 'liver2': [6, 15], 'pancreas': [11]}
Organ_HU = {'liver': [100, 160]}


def grow_tumor(current_state, density_organ_state, kernel_size, steps, all_states, organ_hu_lowerbound, organ_standard_val, outrange_standard_val, threshold, density_organ_map):
    # process
    original_state = current_state.cpu().numpy().copy()
    for i in range(steps+1):
        current_state = update_cellular(current_state, density_organ_state, (kernel_size[0], kernel_size[1], kernel_size[2]), (
            organ_hu_lowerbound, organ_standard_val, outrange_standard_val, threshold)).clamp(max=(outrange_standard_val + 2))
        temp = current_state.cpu().numpy().copy()
        # print(np.sum(temp==0))
        all_states.append(temp)

    all_states = np.array(all_states)

    # postprocess
    all_states[all_states >= outrange_standard_val] = 0
    all_states[all_states >= threshold] = threshold

    # Blur the tumor map
    temp = all_states[steps].astype(np.int16)
    kernel = (3, 3)

    for z in range(temp.shape[0]):
        temp[z] = cv2.GaussianBlur(temp[z], kernel, 0)
        # temp[z] = cv2.filter2D(temp[z], -1, sharpen_kernel)

    # save the tumor map
    all_states[steps] = temp
    save = sitk.GetImageFromArray(all_states[steps])
    sitk.WriteImage(save, './test_result/tumor.nii.gz')

    unique_values, counts = np.unique(all_states[steps], return_counts=True)
    for value, count in zip(unique_values, counts):
        print(f"value: {value} times: {count} ")
    return all_states

    # for i in range(steps):
    #     cpl3d.plot3d(all_states, timestep=i)


def mass_effect(img, tumor, start_point):
    '''
    Mass effect transformation of a image region
    '''
    # preprocessing
    tumor[(tumor > 0)] = 1
    x = start_point[1]
    y = start_point[2]
    for c in range(img.shape[0]):
        if tumor[c, x, y] == 1:
            radius = 0
            while tumor[c, x + radius, y] == 1:
                radius += 1
            print(c, radius)
            img[c] = expand(img[c], y, x, 1.3 * radius, 30) 
                        
                
    return img


def expand(src, PointX, PointY, Radius, Strength):
    '''
    Expansion transformation of a image region
    https://www.jb51.net/article/230354.htm
    Param:
        src (2d ndarray): source image
        PointX (int): center in the X axis
        PointY (int): center in the Y axis
        Radius (int): the radius of expanded area
        Strength (int): the expansion strength, 0-100
    Return:
        processed_image (2d ndarray): expanded image
    '''
    processed_image = np.zeros(src.shape, np.uint8)
    processed_image = src.copy()
    height = src.shape[0]
    width = src.shape[1]
    PowRadius = Radius * Radius

    maskImg = np.zeros(src.shape[:2], np.uint8)
    cv2.circle(maskImg, (PointX, PointY),
               math.ceil(Radius), (255, 255, 255), -1)

    mapX = np.vstack([np.arange(width).astype(
        np.float32).reshape(1, -1)] * height)
    mapY = np.hstack([np.arange(height).astype(
        np.float32).reshape(-1, 1)] * width)

    OffsetX = mapX - PointX
    OffsetY = mapY - PointY
    XY = OffsetX * OffsetX + OffsetY * OffsetY

    ScaleFactor = 1 - XY / PowRadius
    ScaleFactor = 1 - Strength / 100 * ScaleFactor
    UX = OffsetX * ScaleFactor + PointX
    UY = OffsetY * ScaleFactor + PointY
    UX[UX < 0] = 0
    UX[UX >= width] = width - 1
    UY[UY < 0] = 0
    UY[UY >= height] = height - 1

    np.copyto(UX, mapX, where=maskImg == 0)
    np.copyto(UY, mapY, where=maskImg == 0)

    UX = UX.astype(np.float32)
    UY = UY.astype(np.float32)

    processed_image = cv2.remap(src, UX, UY, interpolation=cv2.INTER_LINEAR)

    return processed_image

def Quantify(processed_organ_region, organ_hu_lowerbound, organ_standard_val, outrange_standard_val):
    # Quantify the density of the organ
    interval = (outrange_standard_val - organ_hu_lowerbound) / 3
    processed_organ_region[(processed_organ_region < (
        organ_hu_lowerbound+interval))] = organ_hu_lowerbound
    processed_organ_region[(processed_organ_region >= (organ_hu_lowerbound+interval)) & (
        processed_organ_region < (organ_hu_lowerbound + 2*interval))] = organ_hu_lowerbound + interval
    processed_organ_region[(processed_organ_region >= (organ_hu_lowerbound + 2*interval)) & (
        processed_organ_region < outrange_standard_val)] = organ_hu_lowerbound + 2*interval
    

    density_organ_map = processed_organ_region.copy()

    processed_organ_region[processed_organ_region <
                           outrange_standard_val] = organ_standard_val
    processed_organ_region[processed_organ_region == outrange_standard_val] = 1
    binary_array = processed_organ_region.copy()

    # size
    k = 20000
    # Unicom domain
    labeled_array, num_features = label(
        binary_array, connectivity=3, return_num=True)
    regions = regionprops(labeled_array)
    new_array = np.zeros_like(binary_array)

    # add the region
    for region in regions:
        if region.area > k:
            label_value = region.label
            new_array[labeled_array == label_value] = 1

    processed_organ_region = new_array.copy()

    print(processed_organ_region.shape)
    # save
    

    processed_organ_region[processed_organ_region == 1] = outrange_standard_val

    density_organ_map[(density_organ_map == outrange_standard_val) & (
        processed_organ_region != outrange_standard_val)] = organ_hu_lowerbound + 2*interval
    
  
    
    return processed_organ_region, density_organ_map

def map_to_CT_value(img, tumor, density_organ_map, steps, threshold, outrange_standard_val, organ_hu_lowerbound, organ_standard_val, start_point):

    img = img.astype(np.float32)
    tumor = tumor[steps].astype(np.float32)
    density_organ_map = density_organ_map.astype(np.float32)

    mass_img = img.copy()
    # mass effect
    mass_img = mass_effect(mass_img,  tumor.copy(), start_point)
    
    mass_img[tumor > 0] = img[tumor > 0]
    img = mass_img.copy()
   
    select_organ_region = density_organ_map.copy()
    select_organ_region[ tumor > 0] = 0

    processed_organ_region = img.copy()
    processed_organ_region[select_organ_region >= outrange_standard_val] = outrange_standard_val
    processed_organ_region[processed_organ_region >= outrange_standard_val] = outrange_standard_val
    
    
    processed_organ_region, density_organ_map = Quantify(processed_organ_region, organ_hu_lowerbound, organ_standard_val, outrange_standard_val)
    # density_organ_map = mass_effect(density_organ_map,  tumor[steps], start_point)
    save = sitk.GetImageFromArray(density_organ_map)
    sitk.WriteImage(save, './test_result/mass_soft_map.nii.gz')
    save = sitk.GetImageFromArray(img)
    sitk.WriteImage(save, './test_result/mass_effect.nii.gz')
    
    
    # preprocessing
    # original_mask = density_organ_map.copy()
    # original_mask[original_mask < outrange_standard_val] = 0
    # original_mask[original_mask > 0] = 50
    # target_mask = density_organ_map.copy()
    # target_mask[(tumor > 0) | (target_mask < outrange_standard_val)] = 0
    # target_mask[target_mask > 0] = 50

    # get the deformation field of original mask to target mask
    # use the deformation field to transform the original image

    # temp_deformation = []
    # temp_img = []

    # for i in range(img.shape[0]):
    #     deformation_field = cv2.calcOpticalFlowFarneback(original_mask[i], target_mask[i], None, 0.5, 3, 150, 3, 5, 1.2, 0)
    #     temp_deformation.append(deformation_field)

    #     output_image = img[i]
    #     for y in range(img[i].shape[0]):
    #         for x in range(img[i].shape[1]):
    #             displacement = deformation_field[y, x]
    #             new_x = int(x + displacement[1])
    #             new_y = int(y + displacement[0])

    #             if 0 <= new_x < img[i].shape[1] and 0 <= new_y < img[i].shape[0]:
    #                 # output_image[y, x] = img[i][y, x]
    #                 output_image[new_y, new_x] = img[i][y, x]
    #     temp_img.append(output_image)

    # img = np.array(temp_img)
    # # save
    # deformation_field = np.array(temp_deformation)
    # print(deformation_field.shape)
    # print(deformation_field)
    # print(np.max(deformation_field))
    # print(np.min(deformation_field))
    # save = sitk.GetImageFromArray(deformation_field)
    # sitk.WriteImage(save, './test_result/deformation_field.nii.gz')

    # registration
    # registration_method = sitk.ImageRegistrationMethod()
    # registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    # registration_method.SetMetricSamplingStrategy(registration_method.REGULAR)
    # registration_method.SetMetricSamplingPercentage(0.01)
    # registration_method.SetInterpolator(sitk.sitkLinear)
    # registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    # registration_method.SetOptimizerScalesFromPhysicalShift()
    # initial_transform = sitk.CenteredTransformInitializer(original_mask, target_mask, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY)
    # registration_method.SetInitialTransform(initial_transform, inPlace=False)
    # final_transform = registration_method.Execute(original_mask, target_mask)
    # img = sitk.GetImageFromArray(img)
    # transformed_img = sitk.Resample(img, final_transform, sitk.sitkLinear, 0.0, img.GetPixelID())
    # img = sitk.GetArrayFromImage(transformed_img)



    # deal with the conflict vessel
    interval = (outrange_standard_val - organ_hu_lowerbound) / 3
    vessel_condition = (density_organ_map == outrange_standard_val) & (tumor >= threshold/2)
    vessel_value = np.random.randint(40, 50, tumor.shape, dtype=np.int16)

    # deal with the high intensity tissue
    high_tissue_condition = (density_organ_map == (organ_hu_lowerbound + 2 * interval)) & (tumor != 0)
    high_tissue_value = np.random.randint(20, 30, tumor.shape, dtype=np.int16)

    kernel = (3, 3)
    for z in range(vessel_value.shape[0]):
        vessel_value[z] = cv2.GaussianBlur(vessel_value[z], kernel, 0)
        high_tissue_value[z] = cv2.GaussianBlur(high_tissue_value[z], kernel, 0)

    img[vessel_condition] *= (organ_hu_lowerbound + interval/2) / outrange_standard_val
    img[high_tissue_condition] *= (organ_hu_lowerbound + 2 * interval) / outrange_standard_val

    

    # random tumor value
    tumor_value = np.random.randint(20, 30, tumor.shape, dtype=np.int16)
    tumor_value[tumor == 0] = 0

    # blur the tumor value
    kernel = (3, 3)
    for z in range(tumor_value.shape[0]):
        tumor_value[z] = cv2.GaussianBlur(tumor_value[z], kernel, 0)

    # CT mapping function
    # map_img = img * -(tumor/40 - 1) + tumor/50 * tumor_value

    map_img = img * - (tumor / (threshold + 3 * threshold) - 1) - (1 - tumor/(threshold + 0)) * tumor_value

    # postprocess
    tumor_region = map_img.copy()
    tumor_region[tumor == 0] = 0
    for z in range(tumor_region.shape[0]):
        tumor_region[z] = cv2.GaussianBlur(tumor_region[z], kernel, 0)
    map_img[(tumor >= threshold/2) & (density_organ_map >= (organ_hu_lowerbound + 2 * interval))] = tumor_region[(tumor >= threshold/2) & (density_organ_map >= (organ_hu_lowerbound + 2 * interval))]

    map_img = map_img.astype(np.int16)

    save = sitk.GetImageFromArray(map_img)
    sitk.WriteImage(save, './test_result/output.nii.gz')



def main():

    # parser = argparse.ArgumentParser()
    # ## for distributed training
    # parser.add_argument('--steps', default=25, type=int, help='step')
    # parser.add_argument('--size', default=(50,50,50), type=tuple, help='size of the organ')
    # parser.add_argument('--kernel_size', default=(3,3,3), type=tuple, help='Receptive Field')
    # args = parser.parse_args()

    # Load the image
    img_path = './data/img/FELIX-CYS-1000_VENOUS.nii.gz'
    img = sitk.ReadImage(img_path)
    img = sitk.GetArrayFromImage(img)
    print(img.shape)
    print(img.dtype)
    # Load the mask
    mask_path = './data/mask/pseudo_label.nii.gz'
    mask = sitk.ReadImage(mask_path)
    mask = sitk.GetArrayFromImage(mask)

    # initial
    steps = 20  # step
    size = (50, 50, 50)  # (x,y,z)
    kernel_size = (3, 3, 3)  # Receptive Field
    organ_hu_lowerbound = Organ_HU['liver'][0]  # organ hu lowerbound
    outrange_standard_val = Organ_HU['liver'][1]  # outrange standard value
    organ_standard_val = 0  # organ standard value
    threshold = 10  # threshold

    # load organ and quantify
    # get the organ region
    organ_region = np.where(np.isin(mask, Organ_List['liver']))
    min_x = min(organ_region[0])
    max_x = max(organ_region[0])
    min_y = min(organ_region[1])
    max_y = max(organ_region[1])
    min_z = min(organ_region[2])
    max_z = max(organ_region[2])

    # crop the organ
    cropped_organ_region = mask[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1]
    cropped_img = img[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1]
    print(cropped_organ_region.shape)
    print(cropped_img.shape)

    # Quantify the density of the organ
    select_organ_region = np.isin(cropped_organ_region, Organ_List['liver2'])
    processed_organ_region = cropped_img.copy()
    processed_organ_region[~select_organ_region] = outrange_standard_val
    processed_organ_region[processed_organ_region >
                           outrange_standard_val] = outrange_standard_val

    min_val = np.min(processed_organ_region[select_organ_region])
    max_val = np.max(processed_organ_region[select_organ_region])
    print(min_val, max_val)

    processed_organ_region, density_organ_map = Quantify(processed_organ_region, organ_hu_lowerbound, organ_standard_val, outrange_standard_val)
    save = sitk.GetImageFromArray(density_organ_map)  
    sitk.WriteImage(save, './test_result/test_soft.nii.gz')
    save = sitk.GetImageFromArray(processed_organ_region)
    sitk.WriteImage(save, './test_result/test.nii.gz')

    current_state = torch.tensor(
        processed_organ_region, dtype=torch.int32).cuda(device='cuda:6')
    density_organ_state = torch.tensor(
        density_organ_map, dtype=torch.int32).cuda(device='cuda:6')

    # random select a start point
    matching_indices = np.argwhere(
        processed_organ_region == organ_standard_val)
    if matching_indices.size > 0:
        random_index = np.random.choice(matching_indices.shape[0])
        start_point = matching_indices[random_index]

    current_state[start_point[0], start_point[1],
                  start_point[2]] = threshold/2  # start point initialize
    all_states = []  # states of each step

    # simulate tumor growth
    tumor_out = grow_tumor(current_state, density_organ_state, kernel_size, steps, all_states,
                           organ_hu_lowerbound, organ_standard_val, outrange_standard_val, threshold, density_organ_map)

    # map to CT value
    print(cropped_img.dtype)

    img_out = map_to_CT_value(cropped_img, tumor_out, density_organ_map,
                              steps, threshold, outrange_standard_val, organ_hu_lowerbound, organ_standard_val, start_point)
    
    # save the result
    img[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1] = img_out
    save = sitk.GetImageFromArray(img)
    sitk.WriteImage(save, './test_result/result.nii.gz')
    mask = np.zeros_like(img)
    mask[min_x:max_x+1, min_y:max_y+1, min_z:max_z+1] = tumor_out[steps]
    mask[mask > 0] = 1
    save = sitk.GetImageFromArray(mask)
    sitk.WriteImage(save, './test_result/mask.nii.gz')

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'cellpylib3d'